In [1]:
# -*- coding: utf-8 -*-

import netCDF4 as nc4
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [30]:
""" ********  FILES TO PLOT  *************
    *****************************************
    tenseur divrot - freeslip
    convergence 1°/4, /8, /16, /48
    0°, 45°
"""

pp = "/gpfswork/rech/omr/ukd13rj/plots/"
dirt = "/gpfswork/rech/omr/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/AM98/"
dirm = "/gpfswork/rech/omr/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/AM98/MESHMASK_v081121/"

#A = [[4 , 45. , dirt+"EXP_pm4_fs45/AM98_pm_4_fs_45_1m_00010101_00401230_grid_T.nc",
#                dirm+"EXP_pm4_fs45/mesh_mask_4.nc"],
#     [16 , 45. , dirt+"EXP_pm16_fs45/AM98_pm_16_fs_45_1m_00010101_00401230_grid_T.nc",
#               dirm+"EXP_pm16_fs45/mesh_mask16.nc"]] 

#A = [[4 , 45. , dirt+"EXP_ref4_newsym_fs45/ass0p1/AM98_ref4_newsym_fs45_40y_1m_00010101_00401230_grid_T.nc",
#                dirm+"mesh_mask_n4_t45.nc", 50],
#     [8 , 45. , dirt+"EXP_ref8_newsym_fs45/AM98_ref8_newsym_fs45_40y_1m_00010101_00390608_grid_T.nc",
#                dirm+"mesh_mask_n8_t45.nc", 50],
#     [16, 45. , dirt+"EXP_ref16_newsym_fs45/ass_0p1/AM98_ref16_newsym_fs45_40y_1m_00010101_00401230_grid_T.nc",
#                dirm+"mesh_mask_n16_t45.nc", 50],
#     [48, 45. , dirt+"EXP_ref48_newsym_fs45/Lv500/AM98_ref48_newsym_fs45_40y_1m_00010101_00401230_grid_T.nc",
#                dirm+"mesh_mask_n48_t45.nc", ]] 


A = [[4 , 0. , dirt+"EXP_ref4_fs0/AM98_ref4_fs0_40y_1m_00010101_00401230_grid_T.nc",
               dirm+"mesh_mask_n4_t0.nc",0]]
B = [[4 , 45. , dirt+"EXP_ref4_fs45/AM98_ref4_fs45_1m_00010101_00401230_grid_T.nc",
               dirm+"mesh_mask_n4_t45.nc",0]]
C = [[4 , 0. , dirt+"EXP_ref4_ns0/AM98_ref4_ns0_40y_1m_00010101_00401230_grid_T.nc",
               dirm+"mesh_mask_n4_t0.nc",120]]
D = [[48 , 0. , dirt+"EXP_ref48_fs0/AM98_ref48_fs0_40y_1m_00010101_00401230_grid_T.nc",
               dirm+"mesh_mask_n48_t0.nc",0]]
E = [[48 , 45. , dirt+"EXP_ref48_fs45/AM98_ref48_fs45_40y_1m_00010101_00401230_grid_T.nc",
               dirm+"mesh_mask_n48_t45.nc",0]]


In [70]:
Listdt = [E]

""" *************************** PARAMETERS
"""

NI = 1 ; NJ = 1
L=2000E3 # 2000 km
vmin = 200 ; vmax = 800 ; cticks = np.arange(vmin, vmax+20., 5*20.)
#vmin = 25 ; vmax = 50 ; cticks = np.arange(vmin, vmax+10., 5.)# coriois
f0=0.7E-4 ; beta = 2E-11 ; gp = 0.02

tickszer = [np.float64(i)*L/4. for i in range(5)]

# palette = plt.get_cmap('RdYlBu_r')
palette = plt.get_cmap('RdBu_r', 10)

save = 1 ; psave = pp+"ssh48_45.png" ; pdpi = 200
fig, ax = plt.subplots(NI,NJ, figsize=(NJ*3,NI*3), dpi = pdpi, squeeze=False)

""" *************************** DATA
"""
for i in range(NI):
    for j in range(NJ):
        n,theta,pdtT,pmm,nmean = Listdt[j][i]
        dx = 100E3/np.float64(n)
        # reading
        print("... dataframe[%d,%d], n = %d" % (i,j,n))
        dt = nc4.Dataset(pdtT)
        mm = nc4.Dataset(pmm)
        tmask = mm.variables['tmask'][0,0]
        glamt = mm.variables['glamt'][0] ; gphit = mm.variables['gphit'][0]
        # grid
        lx = dx*np.cos((45+theta)*np.pi/180)/np.sqrt(2) ; ly = dx*np.sin((45+theta)*np.pi/180)/np.sqrt(2)
        nx,ny = np.shape(glamt)
        gridx = np.zeros((nx,ny)) ; gridy = np.zeros((nx,ny))
        gridx = glamt - lx ; gridy = gphit - ly
        # data
        if nmean>0:
            ssh = dt.variables['sossheig'][-nmean:,:,:] +500.
            nT,_,_ = np.shape(ssh)
            data = np.mean(ssh[(nT-nmean):,:,:], axis=0)
            data = np.ma.masked_where(tmask==0,data)
        else :
            ssh = dt.variables['sossheig'][-1,:,:] +500.
            data = np.ma.masked_where(tmask==0,ssh)
        # plot internal raidus
        # data=np.sqrt(gp*ssh)/(f0+beta*gphit)/1E3
        #
        cf = ax[i][j].pcolormesh(gridx, gridy, data,
                            vmin=vmin, vmax=vmax, alpha = 0.9,
                            # levels = levels,
                            cmap = palette)
        # ax[i][j].title.set_text(titles[i][j])
        # contour
        c1= ax[i][j].contour(glamt,gphit, data,
                        levels = cticks,
                        vmin=vmin,vmax=vmax,
                        linewidths =0.3, colors=('k',),linestyles = "solid")
        ax[i][j].clabel(c1, fmt='%3.0f', colors='k', fontsize=6)
        
        
""" *************************** PRETTY
"""
for i in range(NI):
    for j in range(NJ):
        n,t,pdtT,pmm,_ = Listdt[j][i]
        dx = 100E3/np.float64(n)
        limx = dx/2 ; limy = dx/2
        # print(tickszer)
        ax[i][j].set_xlim(-limx,L + limx)
        ax[i][j].set_xticks(tickszer)
        ax[i][j].set_ylim(-limy,L + limy)
        ax[i][j].set_yticks(tickszer)
        ax[i][j].set_xticklabels([]) ; ax[i][j].set_yticklabels([])
        #
        if (i==NI-1):
            ax[i][j].set_xticklabels(["%d" % (x/1E3) for x in tickszer])
            ax[i][j].set_xlabel("X (km)",size=6)
        #
        if (j==0):
            ax[i][j].set_yticklabels(["%d" % (x/1E3) for x in tickszer])
            ax[i][j].set_ylabel("Y (km)", size=6)
        #
        ax[i][j].patch.set_color('0.')
        ax[i][j].xaxis.set_minor_locator(AutoMinorLocator())
        ax[i][j].yaxis.set_minor_locator(AutoMinorLocator())
        ax[i][j].tick_params(axis = "y", which = 'both', width=1., labelsize = 8, pad = 3, left = True, right = True)
        ax[i][j].tick_params(axis = 'x', which = 'both', width=1., labelsize = 8, pad = 3, bottom = True, top = True)
        ax[i][j].tick_params(which='minor',length = 3)
        ax[i][j].tick_params(which='major',length = 4)
        ax[i][j].set_aspect(aspect='equal') # data coordinate 'equal'


fig.subplots_adjust(right=0.8)

""" *************************** COLORBAR
"""

for i in range(NI):
    pos= ax[i][-1].get_position()
    cax = fig.add_axes([pos.x1+0.03, pos.y0, 0.015, pos.y1-pos.y0])
    cbar = fig.colorbar(cf, cax=cax, 
                        orientation = "vertical", extend = 'both')
    cbar.set_ticks(cticks)
    cbar.set_ticklabels(["%dm" % s for s in cticks])
    cbar.ax.tick_params(labelsize = 6)
    #cbar.set_ticklabels(["%dkm" % s for s in cticks])
    cbar.set_label(r"Upper Layer Width - (m)", size=6)

""" *************************** SAVING
"""

if save :
    print("\nsaving : %s" % psave)
    fig.savefig(psave, dpi = pdpi)
    plt.close()
plt.show()




... dataframe[0,0], n = 48

saving : /gpfswork/rech/omr/ukd13rj/plots/ssh48_45.png


In [29]:
import netCDF4 as nc4
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

""" ********  FILES TO PLOT  *************
    *****************************************
    tenseur divrot - freeslip
    convergence 1°/4, /8, /16, /48
    0°, 45°
"""

pp = "/gpfswork/rech/omr/ukd13rj/plots/"
dirt = "/gpfswork/rech/omr/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/AM98/"
dirm = "/gpfswork/rech/omr/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/AM98/MESHMASK_v081121/"

A = [4 , 0. , dirt+"EXP_ref4_fs0/AM98_ref4_fs0_40y_1m_00010101_00401230_grid_T.nc",
               dirt+"EXP_ref4_fs0/AM98_ref4_fs0_40y_1m_00010101_00401230_grid_U.nc",
               dirt+"EXP_ref4_fs0/AM98_ref4_fs0_40y_1m_00010101_00401230_grid_V.nc",
               dirm+"mesh_mask_n4_t0.nc",0]

Listdt = [A]

""" *************************** PARAMETERS
"""

NI = 1 ; NJ = 1
L=2000E3 # 2000 km
vmin = 200 ; vmax = 800 ; cticks = np.arange(vmin, vmax+20., 5*20.)
tickszer = [np.float64(i)*L/4. for i in range(5)]

# palette = plt.get_cmap('RdYlBu_r')
palette = plt.get_cmap('RdBu_r', 10)
save = 1

""" *************************** DATA
"""

n,theta,pdtT,pdtU, pdtV,pmm,nmean = A
dx = 100E3/np.float64(n)
limx = dx/2 ; limy = dx/2
# reading
mm = nc4.Dataset(pmm)
tmask = mm.variables['tmask'][0,0]
glamt = mm.variables['glamt'][0] ; gphit = mm.variables['gphit'][0]
# grid
#lx = dx*np.cos((45+theta)*np.pi/180)/np.sqrt(2) ; ly = dx*np.sin((45+theta)*np.pi/180)/np.sqrt(2)
lx = 0. ; ly = 0. # shading nearest
nx,ny = np.shape(glamt)
gridx = np.zeros((nx,ny)) ; gridy = np.zeros((nx,ny))
gridx = glamt - lx ; gridy = gphit - ly
# quiver time and space
quitime = slice(0,120,3)
quix = slice(0,-1,3) ; quiy = slice(0,-1,3)
# ssh 
dt = nc4.Dataset(pdtT)
ssh = dt.variables['sossheig'][quitime,:,:] +500.
nT,nI,nJ = np.shape(ssh)
for t in range(nT):
    ssh[t] = np.ma.masked_where(tmask==0,ssh[t])
#U 
speedUt = 0.*ssh
dtu = nc4.Dataset(pdtU)
tmp = dtu.variables['ssu'][quitime,:,:] 
for i in range(0,nI-1):
    speedUt[:,i,:] = 0.5*(tmp[:,i,:]+tmp[:,i+1,:]) 
#and V
speedVt = 0.*ssh
dtv = nc4.Dataset(pdtV)
tmp = dtv.variables['ssv'][quitime,:,:] 
for j in range(0,nJ-1):
    speedVt[:,:,j] = 0.5*(tmp[:,:,j]+tmp[:,:,j+1]) 

def animate_sol(zt,zu,zv):
    """ Create animation"""
    from matplotlib import animation
    from IPython.display import HTML
    # quiverstride c'est pour ne pas plotter toutes les flèches
    # zt = zt[::plot_every]
    # zu = zu[::plot_every]
    # zv = zv[::plot_every]
    
    
    fig,ax=plt.subplots(dpi=200)
        
    # surface plot
    cs=ax.pcolormesh(
        gridx, gridy, zt[0], 
        vmin=vmin, vmax=vmax,
        cmap=palette, shading = "nearest")
    
    # quiver plot velocity
    cq=ax.quiver(
        glamt[quix,quiy], gphit[quix,quiy], 
        zu[-1,quix,quiy], zv[-1,quix,quiy], clip_on=True)
    
    # colorbar
    #plt.colorbar(cs, orientation="horizontal",
    #             pad=0.1, shrink=0.8, extend = 'both')
    cbar = fig.colorbar(cs, orientation = "vertical", extend = 'both', pad = 0.1)
    cbar.set_ticks(cticks)
    cbar.set_ticklabels(["%dm" % s for s in cticks])
    cbar.ax.tick_params(labelsize = 6)
    #cbar.set_ticklabels(["%dkm" % s for s in cticks])
    cbar.set_label(r"Upper Layer Width - (m)", size=6)
    
    """ *************************** PRETTY
    """        
    ax.set_xlim(-limx,L + limx)     # nominal
    ax.set_ylim(-limy,L + limy)     #
    ax.set_xticks(tickszer)
    ax.set_yticks(tickszer)
    ax.set_xticklabels([]) ; ax.set_yticklabels([])
    #
    ax.set_xticklabels(["%d" % (x/1E3) for x in tickszer])
    ax.set_xlabel("X (km)",size=6)
    #
    ax.set_yticklabels(["%d" % (x/1E3) for x in tickszer])
    ax.set_ylabel("Y (km)", size=6)
    #
    ax.patch.set_color('0.')
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(axis = "y", which = 'both', width=1., labelsize = 8, pad = 3, left = True, right = True)
    ax.tick_params(axis = 'x', which = 'both', width=1., labelsize = 8, pad = 3, bottom = True, top = True)
    ax.tick_params(which='minor',length = 3)
    ax.tick_params(which='major',length = 4)
    ax.set_aspect(aspect='equal') # data coordinate 'equal'

    fig.subplots_adjust(right=0.8)
    
    def animate(n):
        cs.set_array(zt[n].flatten())
        cq.set_UVC(zu[n,quix,quiy], zv[n,quix,quiy])
        return(cs,cq)

    anim = animation.FuncAnimation(
    fig, animate, frames=len(zt[:,0,0]), interval=200, blit=True,)
    vid = anim.to_html5_video()
    # save
    if save:
        writervideo = animation.FFMpegWriter(fps=60)
        anim.save("ssh.mp4", writer=writervideo)
        print("saved")
    plt.close(fig)
    
    return HTML(vid)

animate_sol(ssh,speedUt,speedVt)

saved
